In [1]:
import pandas as pd
import numpy as np
import string
from bs4 import BeautifulSoup
import unidecode
from word2number import w2n
import contractions
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC


from sklearn import metrics
from numpy import mean
from numpy import std


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics.pairwise import euclidean_distances
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif
from sklearn.feature_selection import VarianceThreshold


In [2]:
train_data = pd.read_csv("Train.csv", sep=',')
train_data.drop(train_data.filter(regex="Unnamed"), axis =1, inplace= True)
train_data.drop(train_data.filter(regex="id"), axis =1, inplace= True)
test_data = pd.read_csv("Test.csv", sep=',')
test_data.drop(train_data.filter(regex="Unnamed"), axis =1, inplace= True)
test_data.drop(train_data.filter(regex="id"), axis =1, inplace= True)


In [3]:
#remove URL and @xxx from text
def to_lower(text):
    text = text.lower()
    return text
def remove_url_and_at(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\S+", "", text)
    return text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove extra whitespaces from text
def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove html tags from text
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#remove accented characters from text
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

# this fuction is from <https://gist.github.com/jiahao87/d57a2535c2ed7315390920ea9296d79f>
#expand contractions in text
def expand_contractions(text):
    #expand shortened words
    text = contractions.fix(text)
    return text

# standardize emoji
def emotion(text):
    text = re.sub(r':\)|:D|: \)|=\)|:-\)', " :) ", text)
    text = re.sub(r':\(|: \(|:-\(', " :(  ", text)                      
    return text

#remove punctuation beside !&?
def remove_punc_and_number(text):
    punctuation = list(string.punctuation)
    punctuation.remove('?')
    punctuation.remove('!')
    for i in range(len(text)):
        if text[i] in punctuation:
            if i+1 <= len(text)-1:
                if (text[i] == ":") and (text[i+1] == "(" or text[i+1] == ")"):
                    continue
            if i >= 1:
                if (text[i] == ")" or text[i] == "(") and (text[i-1] == ':'):
                    continue
            text = text.replace(text[i], ' ')
    text = re.sub(r'[0-9]+', '', text)
    return text


#extract !&?
def create_whitespace(text):
    text = re.sub(r'\?+', ' ? ', text)
    text = re.sub(r'!+', ' ! ', text)              
    return text

#remove words have <=2 length
def remove_shortwords(filtered_sentence):
    return_list=[]
    for word in filtered_sentence:
        if len(word) > 2:
            return_list.append(word)
    return  return_list

#lemitization
def lemmetize_print(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

# remove stop words from text
# exclude words "no","not",from stopwords list
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')

def remove_stopwords(text):
    text_lst = text.split(" ")
    for word in text_lst:
        if word in stop_words:
            text_lst.remove(word)
    return text_lst

def remove_deplicated_charac(text):
    text = re.sub("(.)\\1{2,}", "\\1", text)
    return text

def remove_deplicated_charac2(text):
    text_lst = text.split(" ")
    for i in range(len(text_lst)):
        text_lst[i] = re.sub("^(.)\\1{1,}","\\1" ,text_lst[i])
    return " ".join(text_lst)

In [4]:
#preprocessing text
from collections import Counter
import pycountry


frequency = Counter()
month = ["january", "february", "march", "april", "june", "july", "august", "september", "october", "november","december"]
number =["one","two","three","four","five","six","seven","eight","nine","ten"]
countries= []
for p in pycountry.countries:
    p = p.name.lower()
    countries.append(p)
    
    
def clean_text_instance(df):
    for i in range(len(df["text"])):
        text = df["text"][i]
        text = to_lower(text)
        text = df["text"][i]
        text = contractions.fix(text) 
        text = remove_deplicated_charac(text)
        text = remove_deplicated_charac2(text)
        text = remove_accented_chars(text)
        text = remove_url_and_at(text)
        text = strip_html_tags(text)
        text = emotion(text)
        text = remove_punc_and_number(text)
        text = create_whitespace(text)
        text = remove_whitespace(text)
        text = lemmetize_print(text)

        #word_tokens = word_tokenize(text)
        stop_word_removed_list = remove_stopwords(text) 
        new_filtered_sentence = remove_shortwords(stop_word_removed_list)
        #record number of occurance for unique words
        frequency.update(new_filtered_sentence)
        new_sentence = " ".join(new_filtered_sentence)
        df.iloc[i,df.columns.get_loc('text')] = new_sentence
    #keep only words with more than 2 occurace
    for i in range(len(df["text"])):
        word_list = df["text"][i].split()
        new_word_list = [word for word in word_list if frequency[word] >3]
        new_word_list = [word for word in word_list if frequency[word] <7000]
        new_word_list = [word for word in word_list if word not in month]
        new_word_list = [word for word in word_list if word not in number]
        new_word_list = [word for word in word_list if word not in countries]
        new_sentence = " ".join(new_word_list)
        df.iloc[i,df.columns.get_loc('text')]= new_sentence 

In [5]:
clean_text_instance(test_data)
clean_text_instance(train_data)

In [12]:
#separating instance and label for Train
X_train_raw = [x[0] for x in train_data[['text']].values]
y_train_ori = [x[0] for x in train_data[['sentiment']].values]

#check the result
print("Train length:",len(X_train_raw))

#separating instance and label for Test
X_test_raw = [x[0] for x in test_data[['text']].values]

#check the result
print("Test length:",len(X_test_raw))

Train length: 21802
Test length: 6099


In [13]:
# split the training data into subtraining and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_raw, y_train_ori, test_size = 0.28, random_state = 42)
print(len(y_train))

15697


In [15]:
#Bow
BoW_vectorizer = CountVectorizer(analyzer ='word', ngram_range =(1,2))
#Build the feature set (vocabulary) and vectorise the Tarin dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train,y_train)
#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_val_BoW = BoW_vectorizer.transform(X_val)
print("Train feature space size (using BoW):",X_train_BoW.shape)
print("Test feature space size (using BoW):",X_val_BoW.shape)
print("Dimension of y_train:",len(y_train))
#TFIDF
tfidf_vectorizer = TfidfVectorizer(analyzer ='word', ngram_range =(1,2))
#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train,y_train)
#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_val_tfidf = tfidf_vectorizer.transform(X_val)
print("Train feature space size (using BoW):",X_train_tfidf.shape)
print("Test feature space size (using BoW):",X_val_tfidf.shape)
print("Dimension of y_train:",len(y_train))
    

#remove feature with low variance
threshold = 0.0001
selector = VarianceThreshold(threshold  = threshold)
X_train_BoW_var = selector.fit_transform(X_train_BoW,y_train)
X_val_BoW_var = selector.transform(X_val_BoW)
print("Train feature space size (using BoW):",X_train_BoW_var.shape)
print("Test feature space size (using BoW):",X_val_BoW_var.shape)
print("Dimension of y_train:",len(y_train))
# tfidf
X_train_tfidf_var= selector.fit_transform(X_train_tfidf, y_train)
X_val_tfidf_var = selector.transform(X_val_tfidf)
print("Train feature space size (using tfidf):",X_train_tfidf_var.shape)
print("Test feature space size (using tfidf):",X_val_tfidf_var.shape)
print("Dimension of y_train:",len(y_train))
   

#feature selection select K best feature using BoW
k_best1 = 900
select_k_best = SelectKBest(f_classif, k = k_best1)
X_train_BoW_var_selectk = select_k_best.fit_transform(X_train_BoW_var, y_train)
X_val_BoW_var_selectk = select_k_best.transform(X_val_BoW_var)
print("Train feature space size (using BoW):",X_train_BoW_var_selectk.shape)
print("Test feature space size (using BoW):",X_val_BoW_var_selectk.shape)
    
#feature selection select K best feature using TFIDF
k_best2 = 948
select_k_best = SelectKBest(f_classif, k = k_best2)
X_train_tfidf_var_selectk = select_k_best.fit_transform(X_train_tfidf_var,y_train)
X_val_tfidf_var_selectk = select_k_best.transform(X_val_tfidf_var)
    
#oversampling
oversample = SMOTE()
X_train_BoW_var_selectk, y_train_BoW = oversample.fit_resample(X_train_BoW_var_selectk, y_train)
X_train_tfidf_var_selectk, y_train_tfidf = oversample.fit_resample(X_train_tfidf_var_selectk, y_train)


Train feature space size (using BoW): (15697, 132419)
Test feature space size (using BoW): (6105, 132419)
Dimension of y_train: 15697
Train feature space size (using BoW): (15697, 132419)
Test feature space size (using BoW): (6105, 132419)
Dimension of y_train: 15697
Train feature space size (using BoW): (15697, 21538)
Test feature space size (using BoW): (6105, 21538)
Dimension of y_train: 15697
Train feature space size (using tfidf): (15697, 948)
Test feature space size (using tfidf): (6105, 948)
Dimension of y_train: 15697
Train feature space size (using BoW): (15697, 5000)
Test feature space size (using BoW): (6105, 5000)


In [16]:
#accuracy
SVM_train_acc1 =[]
LR_train_acc1 =[]
MNB_train_acc1 =[]
KNN_train_acc1 =[]
Stacking_train_acc1=[]
SVM_train_acc2 =[]
LR_train_acc2 =[]
MNB_train_acc2 =[]
KNN_train_acc2 =[]
Stacking_train_acc2=[]

#validation accuracy
SVM_valid_acc1 =[]
LR_valid_acc1 =[]
MNB_valid_acc1 =[]
KNN_valid_acc1 =[]
Stacking_valid_acc1=[]
SVM_valid_acc2 =[]
LR_valid_acc2 =[]
MNB_valid_acc2 =[]
KNN_valid_acc2 =[]
Stacking_valid_acc2=[]

#percision
SVM_percision1 =[]
LR_percision1 =[]
MNB_percision1 =[]
KNN_percision1 =[]
SVM_percision_weighted1 =[]
LR_percision_weighted1 =[]
MNB_percision_weighted1 =[]
KNN_percision_weighted1 =[]
Stacking_percision_weighted1=[]

SVM_percision2 =[]
LR_percision2 =[]
MNB_percision2 =[]
KNN_percision2 =[]
SVM_percision_weighted2 =[]
LR_percision_weighted2 =[]
MNB_percision_weighted2 =[]
KNN_percision_weighted2 =[]
Stacking_percision_weighted2=[]

#recall
SVM_recall1 =[]
LR_recall1 =[]
MNB_recall1 =[]
KNN_recall1 =[]
SVM_recall_weighted1 =[]
LR_recall_weighted1 =[]
MNB_recall_weighted1 =[]
KNN_recall_weighted1 =[]
Stacking_recall_weighted1=[]

SVM_recall2 =[]
LR_recall2 =[]
MNB_recall2 =[]
KNN_recall2 =[]
SVM_recall_weighted2 =[]
LR_recall_weighted2 =[]
MNB_recall_weighted2 =[]
KNN_recall_weighted2 =[]
Stacking_recall_weighted2=[]

#F1 score
SVM_F11 =[]
LR_F11 =[]
MNB_F11 =[]
KNN_F11 =[]
Stacking_F11=[]
SVM_F12 =[]
LR_F12 =[]
MNB_F12 =[]
Stacking_F12=[]
KNN_F12 =[]

In [17]:
from sklearn.neighbors import KNeighborsClassifier
"""
    
define the classifiers
    
    
"""
"""define the SVM model"""
SVMclf = SVC(kernel = 'rbf',
                 C = 2,
                 decision_function_shape = 'ovo',
                 class_weight = 'balanced')
SVMclf.fit(X_train_BoW_var_selectk, y_train_BoW)
#train accuracy
SVM_train_acc1.append(SVMclf.score(X_train_BoW_var_selectk, y_train_BoW))
#test accuracy
SVM_valid_acc1.append(SVMclf.score(X_val_BoW_var_selectk, y_val))
#precision
SVM_percision1.append(precision_score(y_val, SVMclf.predict(X_val_BoW_var_selectk), average ="macro"))
SVM_percision_weighted1.append(precision_score(y_val, SVMclf.predict(X_val_BoW_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred1 = SVMclf.predict(X_val_BoW_var_selectk)
SVM_recall1.append(recall_score(y_val, y_val_pred1, average ="macro"))
SVM_recall_weighted1.append(recall_score(y_val, y_val_pred1, average ="weighted"))
#f1
SVM_F11.append(f1_score(y_val, y_val_pred1, average ="weighted"))

# using tfidf
SVMclf.fit(X_train_tfidf_var_selectk,y_train_tfidf)
#train accuracy
SVM_train_acc2.append(SVMclf.score(X_train_tfidf_var_selectk, y_train_tfidf))
#test accuracy
SVM_valid_acc2.append(SVMclf.score(X_val_tfidf_var_selectk, y_val))
#precision
SVM_percision2.append(precision_score(y_val, SVMclf.predict(X_val_tfidf_var_selectk), average ="macro"))
SVM_percision_weighted2.append(precision_score(y_val, SVMclf.predict(X_val_tfidf_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred2 = SVMclf.predict(X_val_tfidf_var_selectk)
SVM_recall2.append(recall_score(y_val, y_val_pred2, average ="macro"))
SVM_recall_weighted2.append(recall_score(y_val, y_val_pred2, average ="weighted"))
#f1
SVM_F12.append(f1_score(y_val, y_val_pred2, average ="weighted"))    
    
    
"""define the logistic regression model"""
LRclf = LogisticRegression(random_state = 42,
                               solver = 'sag', 
                               multi_class = 'multinomial',
                               C = 2,
                               max_iter = 2000,
                               class_weight = 'balanced')
LRclf.fit(X_train_BoW_var_selectk,y_train_BoW)
LR_train_acc1.append(LRclf.score(X_train_BoW_var_selectk, y_train_BoW))
LR_valid_acc1.append(LRclf.score(X_val_BoW_var_selectk, y_val))
#Macro metrics of the classifier
y_val_pred1 = LRclf.predict(X_val_BoW_var_selectk)
LR_percision1.append(precision_score(y_val, y_val_pred1, average ="macro"))   
LR_percision_weighted1.append(precision_score(y_val, y_val_pred1, average ="weighted"))
#recall = tp/(tp+fn)
LR_recall1.append(recall_score(y_val, y_val_pred1, average ="macro"))
LR_recall_weighted1.append(recall_score(y_val, y_val_pred1, average ="weighted"))                                       
#f1
LR_F11.append(f1_score(y_val, y_val_pred1, average ="weighted"))  

# using tfidf
LRclf.fit(X_train_tfidf_var_selectk,y_train_tfidf)
#train accuracy
LR_train_acc2.append(LRclf.score(X_train_tfidf_var_selectk, y_train_tfidf))
#test accuracy
LR_valid_acc2.append(LRclf.score(X_val_tfidf_var_selectk, y_val))
#precision
LR_percision2.append(precision_score(y_val, LRclf.predict(X_val_tfidf_var_selectk), average ="macro"))
LR_percision_weighted2.append(precision_score(y_val, LRclf.predict(X_val_tfidf_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred2 = LRclf.predict(X_val_tfidf_var_selectk)
LR_recall2.append(recall_score(y_val, y_val_pred2, average ="macro"))
LR_recall_weighted2.append(recall_score(y_val, y_val_pred2, average ="weighted"))
#f1
LR_F12.append(f1_score(y_val, y_val_pred2, average ="weighted"))    
                                 

    
"""define a naive bayes model"""
MNBclf= MultinomialNB(alpha =1.2,
                         fit_prior = False)
MNBclf.fit(X_train_BoW_var_selectk,y_train_BoW)
#accuracy
MNB_train_acc1.append(MNBclf.score(X_train_BoW_var_selectk, y_train_BoW))
MNB_valid_acc1.append(MNBclf.score(X_val_BoW_var_selectk, y_val))
#Macro metrics of the classifier
y_val_pred1 = MNBclf.predict(X_val_BoW_var_selectk)
MNB_percision1.append(precision_score(y_val, y_val_pred1, average ="macro"))   
MNB_percision_weighted1.append(precision_score(y_val, y_val_pred1, average ="weighted"))
#recall = tp/(tp+fn)
MNB_recall1.append(recall_score(y_val, y_val_pred1, average ="macro"))
MNB_recall_weighted1.append(recall_score(y_val, y_val_pred1, average ="weighted"))                                       
#f1
MNB_F11.append(f1_score(y_val, y_val_pred1, average ="weighted")) 

# using tfidf
MNBclf.fit(X_train_tfidf_var_selectk,y_train_tfidf)
#train accuracy
MNB_train_acc2.append(MNBclf.score(X_train_tfidf_var_selectk, y_train_tfidf))
#test accuracy
MNB_valid_acc2.append(MNBclf.score(X_val_tfidf_var_selectk, y_val))
#precision
MNB_percision2.append(precision_score(y_val, MNBclf.predict(X_val_tfidf_var_selectk), average ="macro"))
MNB_percision_weighted2.append(precision_score(y_val, MNBclf.predict(X_val_tfidf_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred2 = MNBclf.predict(X_val_tfidf_var_selectk)
MNB_recall2.append(recall_score(y_val, y_val_pred2, average ="macro"))
MNB_recall_weighted2.append(recall_score(y_val, y_val_pred2, average ="weighted"))
#f1
MNB_F12.append(f1_score(y_val, y_val_pred2, average ="weighted"))  

#KNN
KNNclf = KNeighborsClassifier(n_neighbors=5, 
                              weights='distance', 
                              algorithm='auto')
KNNclf.fit(X_train_BoW_var_selectk,y_train_BoW)
#train accuracy
KNN_train_acc1.append(KNNclf.score(X_train_BoW_var_selectk, y_train_BoW))
#test accuracy
KNN_valid_acc1.append(KNNclf.score(X_val_BoW_var_selectk, y_val))
#precision
KNN_percision1.append(precision_score(y_val, KNNclf.predict(X_val_BoW_var_selectk), average ="macro"))
KNN_percision_weighted1.append(precision_score(y_val, KNNclf.predict(X_val_BoW_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred1 = KNNclf.predict(X_val_BoW_var_selectk)
KNN_recall1.append(recall_score(y_val, y_val_pred1, average ="macro"))
KNN_recall_weighted1.append(recall_score(y_val, y_val_pred1, average ="weighted"))
#f1
KNN_F11.append(f1_score(y_val, y_val_pred1, average ="weighted"))

# using tfidf
KNNclf.fit(X_train_tfidf_var_selectk,y_train_tfidf)
#train accuracy
KNN_train_acc2.append(KNNclf.score(X_train_tfidf_var_selectk, y_train_tfidf))
#test accuracy
KNN_valid_acc2.append(KNNclf.score(X_val_tfidf_var_selectk, y_val))
#precision
KNN_percision2.append(precision_score(y_val, KNNclf.predict(X_val_tfidf_var_selectk), average ="macro"))
KNN_percision_weighted2.append(precision_score(y_val, KNNclf.predict(X_val_tfidf_var_selectk), average ="weighted"))
#recall = tp/(tp+fn)
y_val_pred2 = KNNclf.predict(X_val_tfidf_var_selectk)
KNN_recall2.append(recall_score(y_val, y_val_pred2, average ="macro"))
KNN_recall_weighted2.append(recall_score(y_val, y_val_pred2, average ="weighted"))
#f1
KNN_F12.append(f1_score(y_val, y_val_pred2, average ="weighted"))
    
    
#stacking
#estimators =[("svm",SVMclf),("mnb", MNBclf)]
#Stackingclf = StackingClassifier(estimators =estimators, final_estimator = LRclf)
#Stackingclf.fit(X_train_BoW_selectk_var, y_train)
#accuracy
#Stacking_train_acc.append(Stackingclf.score(X_train_BoW_selectk_var, y_train))
#Stacking_valid_acc.append(np.average(cross_val_score(Stackingclf, X_val_BoW_selectk_var, y_val, cv=4)))
#Macro metrics of the classifier
#y_val_pred = Stackingclf.predict(X_val_BoW_selectk_var)  
#Stacking_percision_weighted.append(precision_score(y_val, y_val_pred, average ="weighted"))
#recall = tp/(tp+fn)
#Stacking_recall_weighted.append(recall_score(y_val, y_val_pred, average ="weighted"))                                       #f1
#Stacking_F1.append(f1_score(y_val, y_val_pred, average ="weighted"))

In [18]:
print("___________________________variance threshold = %f____________________________" % threshold)
print("Training accuracy: ")
'''print('______________using BoW k = %d________________'% k_best1)
print("SVM: ", np.mean(SVM_train_acc1),"\n",
      "LR: ",  np.mean(LR_train_acc1),"\n",
      "MNB: ",  np.mean(MNB_train_acc1),"\n",
      "KNN: ",  np.mean(KNN_train_acc1),"\n")'''

print('______________using TFIDF k = %d________________'% k_best2)
print("SVM: ", np.mean(SVM_train_acc2),"\n",
      "LR: ",  np.mean(LR_train_acc2),"\n",
      "MNB: ",  np.mean(MNB_train_acc2),"\n",
     "KNN: ",  np.mean(KNN_train_acc2),"\n")


print("Validation accuracy:")
'''print('______________using BoW k = %d________________'% k_best1)
print(
      "SVM: ", np.mean(SVM_valid_acc1),"\n",
      "LR: ",  np.mean(LR_valid_acc1),"\n",
      "MNB: ",  np.mean(MNB_valid_acc1),"\n",
     "KNN: ",  np.mean(KNN_valid_acc1),"\n")'''
print('______________using TFIDF k = %d________________'% k_best2)
print(
      "SVM: ", np.mean(SVM_valid_acc2),"\n",
      "LR: ",  np.mean(LR_valid_acc2),"\n",
      "MNB: ",  np.mean(MNB_valid_acc2),"\n",
     "KNN: ",  np.mean(KNN_valid_acc2),"\n")

print("Percision:")
print('______________using BoW k = %d________________'% k_best1)
print(
      "SVM: ", np.mean(SVM_percision1),"\n",
      "LR: ",  np.mean(LR_percision1),"\n",
     "MNB: ",  np.mean(MNB_percision1),"\n",
     "KNN: ",  np.mean(KNN_percision1),"\n")
print('______________using TFIDF k = %d________________'% k_best2)
print(
      "SVM: ", np.mean(SVM_percision2),"\n",
      "LR: ",  np.mean(LR_percision2),"\n",
     "MNB: ",  np.mean(MNB_percision2),"\n",
     "KNN: ",  np.mean(KNN_percision2),"\n")

print("Recall:")
print('______________using BoW k = %d________________' % k_best1)
print(
      "SVM: ", np.mean(SVM_recall1),"\n",
      "LR: ",  np.mean(LR_recall1),"\n",
     "MNB: ",  np.mean(MNB_recall1),"\n",
     "KNN: ",  np.mean(KNN_recall1),"\n")
print('______________using TFIDF k = %d________________'% k_best2)
print(
      "SVM: ", np.mean(SVM_recall2),"\n",
      "LR: ",  np.mean(LR_recall2),"\n",
     "MNB: ",  np.mean(MNB_recall2),"\n",
     "KNN: ",  np.mean(KNN_recall2),"\n")

print("Weighted Recall:")
print('______________using BoW k = %d________________'% k_best1)
print(
      "SVM: ", np.mean(SVM_recall_weighted1),"\n",
      "LR: ",  np.mean(LR_recall_weighted1),"\n",
      "MNB: ",  np.mean(MNB_recall_weighted1),"\n",
      "KNN: ", np.mean(KNN_recall_weighted1),"\n")
print('______________using TFIDF k = %d________________'% k_best2)
print(
      "SVM: ", np.mean(SVM_recall_weighted2),"\n",
      "LR: ",  np.mean(LR_recall_weighted2),"\n",
      "MNB: ",  np.mean(MNB_recall_weighted2),"\n",
      "KNN: ", np.mean(KNN_recall_weighted2),"\n")

print("F1:")
print('______________using BoW k = %d________________'% k_best1)
print(
      "SVM: ", np.mean(SVM_F11),"\n",
      "LR: ",  np.mean(LR_F11),"\n",
      "MNB: ",  np.mean(MNB_F11),"\n",
     "KNN: ",  np.mean(KNN_F11),"\n")
print('______________using TFIDF k = %d________________'% k_best2)
print(
      "SVM: ", np.mean(SVM_F12),"\n",
      "LR: ",  np.mean(LR_F12),"\n",
      "MNB: ",  np.mean(MNB_F12),"\n",
     "KNN: ",  np.mean(KNN_F12),"\n")

___________________________variance threshold = 0.000100____________________________
Training accuracy: 
______________using TFIDF k = 948________________
SVM:  0.947678676039836 
 LR:  0.6864015817223199 
 MNB:  0.6404510837727007 
 KNN:  0.997620093731693 

Validation accuracy:
______________using TFIDF k = 948________________
SVM:  0.591973791973792 
 LR:  0.5818181818181818 
 MNB:  0.5171171171171172 
 KNN:  0.47633087633087634 

Percision:
______________using BoW k = 5000________________
SVM:  0.5323996658009956 
 LR:  0.5363346571901587 
 MNB:  0.5422629480183416 
 KNN:  0.4575088323377319 

______________using TFIDF k = 948________________
SVM:  0.5278439478571691 
 LR:  0.5448136882651174 
 MNB:  0.5169327046468454 
 KNN:  0.4678085200483864 

Recall:
______________using BoW k = 5000________________
SVM:  0.5092597379939102 
 LR:  0.547548417489634 
 MNB:  0.5933979918258524 
 KNN:  0.5068033292557113 

______________using TFIDF k = 948________________
SVM:  0.399813705280789 
